In [ ]:
#----------KNN USING TUNNED PARAMETERS-----------

import numpy as np
import pandas as pd
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             f1_score, precision_score, recall_score, cohen_kappa_score,
                             brier_score_loss)
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
import matplotlib.pyplot as plt

# ------------------- LOAD & LABEL DATA -------------------
label_encoder = LabelEncoder()


AttackFree= pd.read_csv('Attack_free new.csv')[0:355409]
DoS= pd.read_csv('DoS_Attack_new.csv')[0:98486]
Fuzzy= pd.read_csv('Fuzzy_Attack_New.csv')[0:88798]
Impersonation= pd.read_csv('Impersonation_Attack_New.csv')[0:149320]

for df in [AttackFree, DoS, Fuzzy, Impersonation]:
    for col in df.columns[1:16]:
        df[col] = label_encoder.fit_transform(df[col].astype(str))

label1 = [1] * len(AttackFree)
label2 = [2] * len(DoS)
label3 = [3] * len(Fuzzy)
label4 = [4] * len(Impersonation)

Dataset = np.concatenate((AttackFree, DoS, Fuzzy, Impersonation))
Labels = np.concatenate((label1, label2, label3, label4))

# ------------------- TRAIN/TEST SPLIT -------------------
X_train, X_test, y_train, y_test = train_test_split(Dataset, Labels, test_size=0.25, random_state=42)

# ------------------- SCALING -------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ------------------- OVERSAMPLING -------------------
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

# ------------------- TRAIN TUNED KNN -------------------
knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance',
    algorithm='auto',
    leaf_size=20,
    p=1
)

start_train = datetime.now()
knn.fit(X_train_bal, y_train_bal)
end_train = datetime.now()
print(f" Training Time: {end_train - start_train}")

# ------------------- TESTING -------------------
start_test = datetime.now()
y_pred = knn.predict(X_test)
end_test = datetime.now()
print(f"Testing Time: {end_test - start_test}")

# ------------------- METRICS -------------------
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']))

# ------------------- CONFUSION MATRIX -------------------
cm = confusion_matrix(y_test, y_pred)
labels = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']

plt.figure(figsize=(7,5))
sns.heatmap(pd.DataFrame(cm, index=labels, columns=labels), annot=True, fmt="d", cmap="Blues")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title("Confusion Matrix - Tuned KNN")
plt.tight_layout()
plt.show()

# ------------------- TPR / TNR -------------------
TPR, TNR = [], []
for i in range(len(cm)):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)
    TPR.append(tp / (tp + fn) if (tp + fn) else 0)
    TNR.append(tn / (tn + fp) if (tn + fp) else 0)

print("True Positive Rate (TPR) and True Negative Rate (TNR) per class:")
for i, cls in enumerate(labels):
    print(f"{cls}:\n TPR = {TPR[i]:.4f}, TNR = {TNR[i]:.4f}")

# ------------------- SUMMARY METRICS -------------------
print("\n Accuracy:", accuracy_score(y_test, y_pred))
print(" Precision:", precision_score(y_test, y_pred, average='weighted'))
print(" Recall:", recall_score(y_test, y_pred, average='weighted'))
print(" F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print(" Cohen Kappa Score:", cohen_kappa_score(y_test, y_pred))

# ------------------- INFERENCE LATENCY -------------------
sample_input = X_test[:1000]
start_inf = time.time()
_ = knn.predict(sample_input)
end_inf = time.time()
latency_ms = (end_inf - start_inf) / len(sample_input) * 1000

print(f"\nInference Time for 1000 samples: {end_inf - start_inf:.4f} sec")
print(f"Average Latency per Sample: {latency_ms:.4f} ms")


